<a href="https://colab.research.google.com/github/Emerald-Z/Colab-Notebooks/blob/main/ISP2022_Recommender_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender Systems

In this exploration I am going to be working with recommender systems, a type of unsupervised machine learning algorithm that is commonly used in popular networks such as Netflix, social media, and search engines. 

# Collaborative filtering

*I ran into some issues here when collecting data because I realized that I asked for favorite movies when instead I should have asked for most recently watched movies or had them rate a set amount of movies. That's why this model gave NaN for a lot of the scoring values with my own data.*

The first technique we're going to take a look at is called Collaborative Filtering, which is also known as User-User Filtering. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the Pearson Correlation Function.

In [ ]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import files

uploaded = files.upload()
uploaded2 = files.upload()

Saving movies.csv to movies.csv


Saving ratings.csv to ratings.csv


In [ ]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('ratings.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('movies.csv')
movies_df.head()

,movieID,title,genre
0,1,Pokémon 2000,adventure/action
1,2,La La Land,Musical
2,3,Jumanji,adventure/action
3,4,The Devil Wears Prada,Drama
4,5,Spirited Away,adventure/action


In [ ]:
movies_df = movies_df.drop('genre',1)
movies_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,movieID,title
0,1,Pokémon 2000
1,2,La La Land
2,3,Jumanji
3,4,The Devil Wears Prada
4,5,Spirited Away


The process for creating a User Based recommendation system is as follows:

Select a user with the movies the user has watched
Based on his rating of the movies, find the top X neighbours
Get the watched movie record of the user for each neighbour
Calculate a similarity score using some formula
Recommend the items with the highest score
Let's begin by creating an input user to recommend movies to:


In [ ]:
userInput = [
            {'title':'Good Will Hunting', 'rating':5},
            {'title':'Harry Potter', 'rating':5},
            {'title':'Spirited Away', 'rating':4},
            {'title':"Home Alone", 'rating':3},
            {'title':'La La Land', 'rating':5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,Good Will Hunting,5
1,Harry Potter,5
2,Spirited Away,4
3,Home Alone,3
4,La La Land,5


#### Add movieId to input user

With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [ ]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)

#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

NameError: ignored

In [ ]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieID'].isin(inputMovies['movieID'].tolist())]
userSubset.head()

,userID,movieID,title,genre,rating
1,2,2.0,La La Land,Musical,5.0
4,5,2.0,La La Land,Musical,5.0
5,6,2.0,La La Land,Musical,5.0
7,8,5.0,Spirited Away,adventure/action,5.0
12,13,2.0,La La Land,Musical,5.0


Group the rows by userID

In [ ]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userID'])

Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.

In [ ]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [ ]:
#the first user
userSubsetGroup[0:3]

[(2,    userID  movieID       title    genre  rating
  1       2      2.0  La La Land  Musical     5.0),
 (5,    userID  movieID       title    genre  rating
  4       5      2.0  La La Land  Musical     5.0),
 (6,    userID  movieID       title    genre  rating
  5       6      2.0  La La Land  Musical     5.0)]

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.
We're going to find out how similar each user is to the input through the Pearson Correlation Coefficient. It is used to measure the strength of a linear association between the two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below.

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y, then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because, for example, two users might rate two series of items totally differently in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

alt text

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation.

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

In [ ]:
userSubsetGroup = userSubsetGroup[0:100]

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient.

In [ ]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieID')
    inputMovies = inputMovies.sort_values(by='movieID')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieID'].isin(group['movieID'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [ ]:
pearsonCorrelationDict.items()

dict_items([(2, 0), (5, 0), (6, 0), (7, 0), (8, 0), (13, 0), (15, 0)])

In [ ]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0,2
1,0,5
2,0,6
3,0,7
4,0,8


top 50 Users that are most similar to the input

In [ ]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
0,0,2
1,0,5
2,0,6
3,0,7
4,0,8


####Recommend movies to this user
We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our pearsonDF from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [ ]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userID', how='inner')
topUsersRating.head()

,similarityIndex,userId,userID,movieID,title,genre,rating
0,0,2,2,2.0,La La Land,Musical,5.0
1,0,2,2,21.0,Lady Bird,Drama,5.0
2,0,2,2,41.0,Dead Poets Society,Drama,5.0
3,0,2,2,47.0,White Chicks,comedy,5.0
4,0,2,2,80.0,Gifted,drama,5.0


Now all we need to do is simply multiply the movie rating by its weight (the similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [ ]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,userID,movieID,title,genre,rating,weightedRating
0,0,2,2,2.0,La La Land,Musical,5.0,0.0
1,0,2,2,21.0,Lady Bird,Drama,5.0,0.0
2,0,2,2,41.0,Dead Poets Society,Drama,5.0,0.0
3,0,2,2,47.0,White Chicks,comedy,5.0,0.0
4,0,2,2,80.0,Gifted,drama,5.0,0.0


In [ ]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieID').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieID,,
2.0,0,0.0
4.0,0,0.0
5.0,0,0.0
9.0,0,0.0
11.0,0,0.0


In [ ]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieID'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieID
movieID,,
2.0,NaN,2.0
4.0,NaN,4.0
5.0,NaN,5.0
9.0,NaN,9.0
11.0,NaN,11.0


In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieID
movieID,,
2.0,NaN,2.0
4.0,NaN,4.0
5.0,NaN,5.0
9.0,NaN,9.0
11.0,NaN,11.0
20.0,NaN,20.0
21.0,NaN,21.0
23.0,NaN,23.0
24.0,NaN,24.0


In [ ]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

KeyError: ignored

# Content-Based Filtering

In [ ]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Storing the movie information into a pandas dataframe
movies_df2 = pd.read_csv('ratings.csv')
#Storing the user information into a pandas dataframe
ratings_df2 = pd.read_csv('movies.csv')
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df2.head()

,movieID,title,genre
0,1,Pokémon 2000,adventure/action
1,2,La La Land,Musical
2,3,Jumanji,adventure/action
3,4,The Devil Wears Prada,Drama
4,5,Spirited Away,adventure/action


In [ ]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df2['genre'] = movies_df2.genre.str.split('|')
movies_df2.head()

,movieID,title,genre
0,1,Pokémon 2000,[adventure/action]
1,2,La La Land,[Musical]
2,3,Jumanji,[adventure/action]
3,4,The Devil Wears Prada,[Drama]
4,5,Spirited Away,[adventure/action]


Since keeping genres in a list format isn't optimal for the content-based recommendation system technique, we will use the One Hot Encoding technique to convert the list of genres to a vector where each column corresponds to one possible value of the feature. This encoding is needed for feeding categorical data. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't. Let's also store this dataframe in another variable since genres won't be important for our first recommendation system.

In [ ]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df2.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df2.iterrows():
    for g in row['genre']:
        moviesWithGenres_df.at[index, g] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieID,title,genre,adventure/action,Musical,Drama,horror/thriller,Romance,comedy,studio ghibli,Feel good,musical,Mystery,romantic/comedy,drama,Disney
0,1,Pokémon 2000,[adventure/action],1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,La La Land,[Musical],0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Jumanji,[adventure/action],1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,The Devil Wears Prada,[Drama],0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Spirited Away,[adventure/action],1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now, let's take a look at how to implement Content-Based or Item-Item recommendation systems. This technique attempts to figure out what a user's favourite aspects of an item is, and then recommends items that present those aspects. In our case, we're going to try to figure out the input's favorite genres from the movies and ratings given.

Using same user as above

In [ ]:
#Filtering out the movies by title
inputId = movies_df2[movies_df2['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)

#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieID,title,genre,rating
0,2,La La Land,[Musical],5
1,5,Spirited Away,[adventure/action],4
2,11,Home Alone,[comedy],3
3,84,Home Alone,[comedy],3


In [ ]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieID'].isin(inputMovies['movieID'].tolist())]
userMovies

,movieID,title,genre,adventure/action,Musical,Drama,horror/thriller,Romance,comedy,studio ghibli,Feel good,musical,Mystery,romantic/comedy,drama,Disney
1,2,La La Land,[Musical],0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Spirited Away,[adventure/action],1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,11,Home Alone,[comedy],0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83,84,Home Alone,[comedy],0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We'll only need the actual genre table, so let's clean this up a bit by 
resetting the index and dropping the movieId, title, genres and year columns.

In [ ]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieID', 1).drop('title', 1).drop('genre', 1)
userGenreTable

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,adventure/action,Musical,Drama,horror/thriller,Romance,comedy,studio ghibli,Feel good,musical,Mystery,romantic/comedy,drama,Disney
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we're ready to start learning the input's preferences!

To do this, we're going to turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling the Pandas "dot" function.

In [ ]:
inputMovies['rating']

0    5
1    4
2    3
3    3
Name: rating, dtype: int64

In [ ]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

adventure/action    4.0
Musical             5.0
Drama               0.0
horror/thriller     0.0
Romance             0.0
comedy              6.0
studio ghibli       0.0
Feel good           0.0
musical             0.0
Mystery             0.0
romantic/comedy     0.0
drama               0.0
Disney              0.0
dtype: float64

Now, we have the weights for every of the user's preferences. This is known as the User Profile. Using this, we can recommend movies that satisfy the user's preferences.

Let's start by extracting the genre table from the original dataframe:

In [ ]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieID'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieID', 1).drop('title', 1).drop('genre', 1)
genreTable.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,adventure/action,Musical,Drama,horror/thriller,Romance,comedy,studio ghibli,Feel good,musical,Mystery,romantic/comedy,drama,Disney
movieID,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.

In [ ]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieID
1    0.266667
2    0.333333
3    0.266667
4    0.000000
5    0.266667
dtype: float64

In [ ]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

movieID
66    0.4
77    0.4
56    0.4
53    0.4
70    0.4
dtype: float64

Here is the final Recommendation table!

In [ ]:
#The final recommendation table
movies_df2.loc[movies_df2['movieID'].isin(recommendationTable_df.head(20).keys())]

,movieID,title,genre
1,2,La La Land,[Musical]
10,11,Home Alone,[comedy]
11,12,Pitch Perfect,[Musical]
19,20,Despicable Me,[comedy]
21,22,Sing 2,[Musical]
26,27,Kung fu panda 1,[comedy]
31,32,Jumanji,[comedy]
34,35,Cats,[Musical]
39,40,Legally Blonde,[comedy]
46,47,White chicks,[comedy]
